#Import Libraries

In [30]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer # technique in Scikit-learn that fills in missing values in a dataset using the k-nearest neighbors (KNN) algorithm.

- Read Data Set

In [31]:
data = pd.read_csv(r"/content/complaints.csv")

# Explore Data

- First Few Rows

In [32]:
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2022-11-19,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"EQUIFAX, INC.",TX,78541.0,Servicemember,NaN,Web,2022-11-19,In progress,Yes,NaN,6222374.0
1,2022-10-13,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the fair credit reporting a...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",GA,30043.0,NaN,Consent provided,Web,2022-10-13,Closed with explanation,Yes,NaN,6079679.0
2,2022-10-13,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,This is not a duplicate nor is this complaint ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,90660.0,NaN,Consent provided,Web,2022-10-13,Closed with non-monetary relief,Yes,NaN,6076990.0
3,2022-10-12,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Personal information incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,19148.0,NaN,Consent not provided,Web,2022-10-12,Closed with non-monetary relief,Yes,NaN,6080312.0
4,2022-11-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",OH,44130.0,NaN,NaN,Web,2022-11-19,In progress,Yes,NaN,6222634.0


# Dataset Explained
- This dataset contains consumer complaints related to financial products and services. It includes details such as the date received, the product and sub-product involved, the issue and sub-issue described by the consumer, and a narrative (if provided). It also tracks the company's response, whether it was timely, and if the consumer disputed it. Additional fields include the state, ZIP code, submission method, and public response from the company. Each complaint is assigned a unique ID, making it useful for trend analysis, customer sentiment tracking, and evaluating company responses.




- Columns Type

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31653 entries, 0 to 31652
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 31653 non-null  object 
 1   Product                       31653 non-null  object 
 2   Sub-product                   31653 non-null  object 
 3   Issue                         31652 non-null  object 
 4   Sub-issue                     30237 non-null  object 
 5   Consumer complaint narrative  2177 non-null   object 
 6   Company public response       4760 non-null   object 
 7   Company                       31652 non-null  object 
 8   State                         31629 non-null  object 
 9   ZIP code                      31616 non-null  float64
 10  Tags                          2628 non-null   object 
 11  Consumer consent provided?    5605 non-null   object 
 12  Submitted via                 31652 non-null  object 
 13  D

- Check for nulls

In [34]:
data.isnull().sum()

,0
Date received,0
Product,0
Sub-product,0
Issue,1
Sub-issue,1416
Consumer complaint narrative,29476
Company public response,26893
Company,1
State,24
ZIP code,37


# Cleaning Data
 - Check For Duplicates

In [35]:
data.duplicated().sum() # no duplicates

0

- Drop Unnecesary Columns

In [36]:
data.drop(["ZIP code","Tags","Consumer consent provided?"], axis=1, inplace=True)
list(data.columns)

['Date received',
 'Product',
 'Sub-product',
 'Issue',
 'Sub-issue',
 'Consumer complaint narrative',
 'Company public response',
 'Company',
 'State',
 'Submitted via',
 'Date sent to company',
 'Company response to consumer',
 'Timely response?',
 'Consumer disputed?',
 'Complaint ID']

- Fix Data types  
  - (obj > datetime)

In [37]:
data['Date received'] = pd.to_datetime(data['Date received'])
data['Date sent to company'] = pd.to_datetime(data['Date sent to company'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31653 entries, 0 to 31652
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Date received                 31653 non-null  datetime64[ns]
 1   Product                       31653 non-null  object        
 2   Sub-product                   31653 non-null  object        
 3   Issue                         31652 non-null  object        
 4   Sub-issue                     30237 non-null  object        
 5   Consumer complaint narrative  2177 non-null   object        
 6   Company public response       4760 non-null   object        
 7   Company                       31652 non-null  object        
 8   State                         31629 non-null  object        
 9   Submitted via                 31652 non-null  object        
 10  Date sent to company          31652 non-null  datetime64[ns]
 11  Company response to consumer

- check nulls in  ***Company public response*** column

In [38]:
data["Company public response"] #There is NaN fill it with 'Pending to provide a public response'

,Company public response
0,NaN
1,Company has responded to the consumer and the ...
2,Company has responded to the consumer and the ...
3,Company has responded to the consumer and the ...
4,NaN
...,...
31648,Company has responded to the consumer and the ...
31649,Company has responded to the consumer and the ...
31650,Company has responded to the consumer and the ...
31651,Company has responded to the consumer and the ...


- Fill NaN values in ***Company public response*** column



In [39]:
data["Company public response"].fillna('Pending to provide a public response', inplace = True)

In [40]:
data["Company public response"]

,Company public response
0,Pending to provide a public response
1,Company has responded to the consumer and the ...
2,Company has responded to the consumer and the ...
3,Company has responded to the consumer and the ...
4,Pending to provide a public response
...,...
31648,Company has responded to the consumer and the ...
31649,Company has responded to the consumer and the ...
31650,Company has responded to the consumer and the ...
31651,Company has responded to the consumer and the ...


In [41]:
data['Sub-product']

,Sub-product
0,Credit reporting
1,Credit reporting
2,Credit reporting
3,Credit reporting
4,Credit reporting
...,...
31648,Credit reporting
31649,Other debt
31650,General-purpose credit card or charge card
31651,Credit reporting


- Fill Nan values in ***Sub-product*** & ***Sub-issue*** columns

In [42]:
data['Sub-product'].fillna("I do not know", inplace=True)
data['Sub-issue'].fillna("I do not know", inplace=True)

<ipython-input-42-55731fcb2111>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Sub-issue'].fillna("I do not know", inplace=True)


Fill Nan values in ***State***  column with the most fequent row

In [43]:
most_freq = data["State"].mode()[0] # CA
data["State"].fillna(most_freq, inplace=True)

<ipython-input-43-f9462f7ea3ca>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["State"].fillna(most_freq, inplace=True)


- Again check for nulls

In [44]:
data.isnull().sum()

,0
Date received,0
Product,0
Sub-product,0
Issue,1
Sub-issue,0
Consumer complaint narrative,29476
Company public response,0
Company,1
State,0
Submitted via,1


- Fill NaN values in ***Consumer complaint narrative*** column

In [45]:
data["Consumer complaint narrative"].fillna("I do not know", inplace= True)

<ipython-input-45-8e33e813c14b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Consumer complaint narrative"].fillna("I do not know", inplace= True)


In [46]:
data["Consumer disputed?"]

,Consumer disputed?
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
31648,NaN
31649,NaN
31650,NaN
31651,NaN


## Using KNNImputer To fill the Nan value with the matching neighbor values


In [47]:
# Replace the category values with Numerical to impute missing Values
data['Consumer disputed?'] = data['Consumer disputed?'].map({'Yes':1,'No': 0})

# Select the column to impute
column_to_impute = 'Consumer disputed?'

# Create a copy of the dataset with the selected column
impute_data = data[[column_to_impute]].copy()

# Create an instance of the KNNImputer with the desired parameters
imputer = KNNImputer(n_neighbors=5)

# Fit and transform the selected column using the imputer
impute_data[column_to_impute] = imputer.fit_transform(impute_data[[column_to_impute]])

# Replace the original column in the dataset with the imputed values
data[column_to_impute] = impute_data[column_to_impute]

# Creating a dictionary for replacement
replacement_dict = {value: 'yes' if value >= 0.5 else 'no' for value in data['Consumer disputed?'].unique()}

# Applying the replacement to the 'Consumer_disputed?' column
data['Consumer disputed?'] = data['Consumer disputed?'].map(replacement_dict)

In [48]:
data["Consumer disputed?"]

,Consumer disputed?
0,no
1,no
2,no
3,no
4,no
...,...
31648,no
31649,no
31650,no
31651,no


- Check the data for last time

In [49]:
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2022-11-19,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I do not know,Pending to provide a public response,"EQUIFAX, INC.",TX,Web,2022-11-19,In progress,Yes,no,6222374.0
1,2022-10-13,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the fair credit reporting a...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",GA,Web,2022-10-13,Closed with explanation,Yes,no,6079679.0
2,2022-10-13,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,This is not a duplicate nor is this complaint ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,Web,2022-10-13,Closed with non-monetary relief,Yes,no,6076990.0
3,2022-10-12,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Personal information incorrect,I do not know,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,Web,2022-10-12,Closed with non-monetary relief,Yes,no,6080312.0
4,2022-11-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I do not know,Pending to provide a public response,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",OH,Web,2022-11-19,In progress,Yes,no,6222634.0


# Save new Data Set

In [50]:
# Save the Datast to use in Power BI Dashboard
data.to_csv('Financial_Consumer_data.csv')